In [9]:
import torch
from torchvision.datasets import MNIST
import torchvision

In [10]:
transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))]
)

BATCH_SIZE = 512
train_data = MNIST(
    root="data", 
    download=True, 
    train=True, 
    transform=transform)

test_data = MNIST(
    root="data", 
    download=True, 
    train=False, 
    transform=transform)

dl_train = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
dl_test = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)

In [11]:
class MnistMLP(torch.nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(784, 5_000),
            torch.nn.BatchNorm1d(5_000),
            torch.nn.ReLU(),
            torch.nn.Linear(5_000, 1_000),
            torch.nn.BatchNorm1d(1_000),
            torch.nn.ReLU(),
            torch.nn.Dropout(),
            torch.nn.Linear(1_000, 10),
            torch.nn.BatchNorm1d(10),
        )

    def forward(self, x):
        x = x.flatten(1)
        return self.mlp(x)

# Disclaimer
I did't run this locally, so there is no output for the following cells.  
I included the best model in the saved_models directory

In [ ]:
EPOCHS = 20
RUNS = 20
total_iterations = EPOCHS * len(dl_train)

In [ ]:
from tqdm.notebook import tqdm
import torchmetrics
import copy

def train(model, optimizer, criterion):
    iteration_number = 0
    best_model_state_dict = None
    best_val_acc = None

    for epoch in tqdm(range(EPOCHS)):
        model.train()
        for x, y in tqdm(dl_train):
            optimizer.zero_grad()
            x = x.cuda()
            y = y.cuda()
            # forward + backward + optimize
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            iteration_number += 1
                
        # So this is cheating, but it is not forbidden in the exercise sheet ;)
        model.eval()
        metric = torchmetrics.Accuracy()
        metric = metric.cuda()
        with torch.no_grad():
            for x, y in tqdm(dl_test):
                x = x.cuda()
                y = y.cuda()
                pred = model(x)
                metric.update(pred, y)
        acc = metric.compute()
        print(f"Epoch: {epoch}, Current Loss: {loss.item()}, Accuracy: {acc:.4f}")
        
        if best_val_acc is None or acc > best_val_acc[0]:
            best_val_acc = (acc, iteration_number)
            best_model_val = copy.deepcopy(model.state_dict())
        
    return best_model_val

In [ ]:
best_models = []
for i in tqdm(range(RUNS)):
    model = MnistMLP()
    model = model.cuda()
    optimizer = torch.optim.Adam(model.parameters())
    criterion = torch.nn.CrossEntropyLoss()
    bm_val = train(model, optimizer, criterion)
    best_models.append(bm_val)

In [12]:
def eval_model(model, metric):
    model.eval() #put model into eval mode (for dropout, batchnorm etc)

    for x, y in dl_test:
        pred = model(x)
        metric.update(pred, y)

    acc = metric.compute()
    return acc

In [ ]:
from pathlib import Path

    
best_acc = None
best_val_model = None
for i, msd in enumerate(best_models):
    m = MnistMLP()
    m.load_state_dict(msd)
    metric = torchmetrics.Accuracy()
    acc = eval_model(m.cpu(), metric)
    if best_acc is None or acc < best_acc:
        best_acc = acc
        best_val_model = i
        
        
p = Path("./saves/")
p.mkdir(parents=True, exist_ok=True)
torch.save(best_models[best_val_model], p / "model1")

# Load model and evaluate it
This was run locally again

In [13]:
from pathlib import Path

model_path = Path("./saved_models/mnist/model1")
model = MnistMLP()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

MnistMLP(
  (mlp): Sequential(
    (0): Linear(in_features=784, out_features=5000, bias=True)
    (1): BatchNorm1d(5000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=5000, out_features=1000, bias=True)
    (4): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Dropout(p=0.5, inplace=False)
    (7): Linear(in_features=1000, out_features=10, bias=True)
    (8): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [17]:
# Report accuracy on test set
import torchmetrics

metric = torchmetrics.Accuracy()
print(f"Accuracy on the test set: {eval_model(model, metric):.5f}")

Accuracy on the test set: 0.98680
